In [1]:
# 普通下载方式

In [2]:
# flag.py

In [3]:
import time
import os
import sys

import requests

POP_CC = ('CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'Downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open (path ,'wb') as f:
        f.write(img)

def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc= cc.lower())
    resp = requests.get(url)
    
    return resp.content

def show(text):
    print(text, end =" ")
    sys.stdout.flush()
    
def download_many(cc_list):
    for cc in sorted(cc_list):
        flag = get_flag(cc)
        show(cc)
        save_flag(flag, cc.lower()+'.gif')
    
    return len(cc_list)

def main(download):
    t0 = time.time()
    counts = download(POP_CC)
    interval = time.time() - t0
    msg = '{} flags hava been loaded in {:.2f}s'
    print(msg.format(counts, interval))

In [4]:
main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 20 flags hava been loaded in 17.91s


In [5]:
# flags_threadpool

In [6]:
from concurrent import futures

MAX_WORKS = 20 # 最多使用的线程

def download_one(cc):
    image = get_flag(cc)
    show(text=cc)
    save_flag(image, cc.lower()+'.gif')

def download_many(cc_list):
    workers = min(len(cc_list), MAX_WORKS)
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    
    return len(list(res))

In [7]:
main(download_many)

NG ID TR BD JPIN DE VN CD PH  US RU BR FR CN MX ET EG IR PK 20 flags hava been loaded in 3.13s


In [11]:
# flags_threadpool_ac.py
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
        
    return len(results)

In [12]:
main(download_many)

Scheduled for BR: <Future at 0x21eb481fe48 state=running>
Scheduled for CN: <Future at 0x21eb481fd30 state=running>
Scheduled for ID: <Future at 0x21eb481f7b8 state=running>
Scheduled for IN: <Future at 0x21eb482f780 state=pending>
Scheduled for US: <Future at 0x21eb482f630 state=pending>
ID <Future at 0x21eb481f7b8 state=finished returned NoneType> result: None
CN <Future at 0x21eb481fd30 state=finished returned NoneType> result: None
BR <Future at 0x21eb481fe48 state=finished returned NoneType> result: None
IN <Future at 0x21eb482f780 state=finished returned NoneType> result: None
US <Future at 0x21eb482f630 state=finished returned NoneType> result: None
5 flags hava been loaded in 5.46s


In [13]:
# 将ProcessPoolExecutor变为ThreadPoolExecutor, 线程数默认为电脑的CPU数

In [24]:
def download_many(cc_list):
    with futures.ProcessPoolExecutor() as executor:
        res = executor.map(download_one, sorted(cc_list))
        
    return len(list(res))

In [25]:
main(download_many)

None flags hava been loaded in 0.23s


In [23]:
from time import sleep ,strftime
from concurrent import futures

def display(*args):
    print(strftime("[%H:%M:%S]"), end=" ")
    print(*args)
    
def loiter(n):
    msg = "{}loiter({}): doing nothing for {}s"
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = "{}loiter({}): done"
    display(msg.format('\t'*n, n))
    return n*10

def main():
    display("Script starting")
    executer = futures.ThreadPoolExecutor(max_workers=3)
    results = executer.map(loiter, range(5))
#     display('results: ', results) # 说明result为生成器
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))

In [24]:
main()

[20:54:11] Script starting
[20:54:11] loiter(0): doing nothing for 0s
[20:54:11] loiter(0): done
[20:54:11] 	loiter(1): doing nothing for 1s
[20:54:11] 		loiter(2): doing nothing for 2s
[20:54:11] [20:54:11] 			loiter(3): doing nothing for 3s
Waiting for individual results:
[20:54:11] result 0: 0
[20:54:12] 	loiter(1): done
[20:54:12] [20:54:12] result 1: 10
				loiter(4): doing nothing for 4s
[20:54:13] 		loiter(2): done
[20:54:13] result 2: 20
[20:54:14] 			loiter(3): done
[20:54:14] result 3: 30
[20:54:16] 				loiter(4): done
[20:54:16] result 4: 40


In [33]:
# tqdm test
from time import sleep
from tqdm import tqdm

for i in tqdm(range(100)):
    sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.83it/s]


In [36]:
for i in tqdm):
    pass

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2785284.35it/s]
